In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\User\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\User\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecat

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [2]:
from transformers import TrainingArguments
training_args = TrainingArguments("test-trainer")

In [3]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
from transformers import Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator,
    tokenizer = tokenizer
)

In [5]:
trainer.train()

C:\Users\User\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Step,Training Loss
500,0.502400
1000,0.275500


TrainOutput(global_step=1377, training_loss=0.3179719906919835, metrics={'train_runtime': 123.7155, 'train_samples_per_second': 88.946, 'train_steps_per_second': 11.13, 'total_flos': 405114969714960.0, 'train_loss': 0.3179719906919835, 'epoch': 3.0})

In [10]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)
print(predictions.predictions, predictions.label_ids)

(408, 2) (408,)
[[-3.7786803   3.2362819 ]
 [ 3.2584288  -2.6263905 ]
 [ 1.674777   -1.0399925 ]
 [-3.8243372   3.2714367 ]
 [ 2.765249   -2.1202352 ]
 [-3.7535908   3.2286727 ]
 [-3.6924222   3.167512  ]
 [-3.8677955   3.318034  ]
 [-3.708501    3.1671546 ]
 [-3.8080177   3.24639   ]
 [-3.8637123   3.3223882 ]
 [ 3.1851928  -2.5798717 ]
 [ 3.5699568  -2.8527176 ]
 [-3.7752988   3.22094   ]
 [-3.8836274   3.3177032 ]
 [ 3.0007093  -2.2493417 ]
 [-3.9043114   3.3506255 ]
 [ 3.1559184  -2.6249375 ]
 [-3.9050725   3.3512564 ]
 [ 2.6343267  -1.8383982 ]
 [ 3.1348684  -2.5518591 ]
 [-2.124333    1.511873  ]
 [ 2.9331794  -2.359104  ]
 [-3.8007708   3.2812412 ]
 [-3.8769336   3.3202677 ]
 [ 0.32411978 -0.04353369]
 [-3.2727222   2.7303233 ]
 [-3.8877792   3.3340206 ]
 [-3.7160182   3.2072048 ]
 [-3.8515077   3.3155713 ]
 [-0.9120052   0.678909  ]
 [-3.8925688   3.331471  ]
 [-3.4792688   2.8983204 ]
 [-3.8385348   3.2867157 ]
 [-3.8444712   3.2985482 ]
 [-3.7566388   3.2441218 ]
 [ 3.4304955

In [11]:
import numpy as np
preds = np.argmax(predictions.predictions, axis=-1)

In [12]:
import evaluate
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8529411764705882, 'f1': 0.8958333333333334}

In [13]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, lables = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references= lables)

In [14]:
training_args = TrainingArguments("test-trainer", evaluation_strategy = "epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

C:\Users\User\anaconda3\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.465118,0.779412,0.859813
2,0.531100,0.466259,0.848039,0.893836
3,0.334700,0.614068,0.852941,0.897260


TrainOutput(global_step=1377, training_loss=0.37226222196868475, metrics={'train_runtime': 116.4343, 'train_samples_per_second': 94.508, 'train_steps_per_second': 11.826, 'total_flos': 405114969714960.0, 'train_loss': 0.37226222196868475, 'epoch': 3.0})